Homework 1: Image Processing
==========

> **Submission Instructions:** Before the deadline, export the completed notebook to PDF and upload it to GradeScope. The PDF should clearly show your code, and the result of running the code. Check the PDF to ensure that it is readable, the font-size is not small, and no information is cut-off. There will be no make-ups or extensions for corrupted/damaged/unreadable PDFs.

**Names of Collaborators**:

The below commands will download the images needed for this problem set. Make sure you run it before you get started.

In [ ]:
!wget -qN https://www.cs.columbia.edu/~vondrick/class/coms4732/hw1/noisy_image.jpg
!wget -qN https://www.cs.columbia.edu/~vondrick/class/coms4732/hw1/edge_detection_image.jpg
!wget -qN https://www.cs.columbia.edu/~vondrick/class/coms4732/hw1/cat.jpg
!wget -qN https://www.cs.columbia.edu/~vondrick/class/coms4732/hw1/dog.jpg

Problem 1: Image Denoising
==========================

Taking pictures at night is challenging because there is less light that hits the film or camera sensor. To still capture an image in low light, we need to change our camera settings to capture more light. One way is to increase the exposure time, but if there is motion in the scene, this leads to blur. Another way is to use sensitive film that still responds to low intensity light. However, the trade-off is that this higher sensitivity increases the amount of noise captured, which often shows up as grain on photos. In this problem, your task is to clean up the noise with signal processing.



Visualizing the Grain
---------------------
To start off, let's load up the image and visualize the image we want to denoise.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython import display
from scipy.signal import convolve2d
from math import *
import time
%matplotlib inline

plt.rcParams['figure.figsize'] = [7, 7]

def load_image(filename):
    img = np.asarray(Image.open(filename))
    img = img.astype("float32") / 255.
    return img

def gray2rgb(image):
    return np.repeat(np.expand_dims(image, 2), 3, axis=2)

def show_image(img):
    if len(img.shape) == 2:
        img = gray2rgb(img)
    plt.imshow(img, interpolation='nearest')

# load the image
im = load_image('noisy_image.jpg')
im = im.mean(axis=2) # convert to grayscale
show_image(im)

Problem 1a: Mean Filter using "for" loop
-----------------------------------------

Let's try to remove this grain with a mean filter. For every pixel in the image, we want to take an average (mean) of the neighboring pictures. Implement this operation using "for" loops and visualize the result:

In [ ]:
im_pad = np.pad(im, 5, mode='constant')  # pad the border of the original image
im_out = np.zeros_like(im)  # initialize the output image array

''' TODO: Implement a mean filter using "for" loop here (modify the im_out matrix). '''

# Define kernel size
kernel_size = 11  # 5-pixel padding means a 11x11 kernel
half_k = kernel_size // 2

# Apply mean filter
for i in range(im.shape[0]):
    for j in range(im.shape[1]):
        im_out[i, j] = np.mean(im_pad[i:i + kernel_size, j:j + kernel_size])


show_image(im_out)

Problem 1b: Implement the `convolve_image` function.
-----------

Convolution provides a mathematical way to apply filters to image. Implement the `convolve_image` function below using `for` loops. Your function should accept an image and a filter matrix, and return the result of convoling the image with the given filter matrix. Note: You cannot use a built-in convolution routine for this problem.

In [ ]:
def convolve_image(image, filter_matrix):
    ''' Convolve a 2D image using the filter matrix.
    Args:
        image: a 2D numpy array.
        filter_matrix: a 2D numpy array.
    Returns:
        the convolved image, which is a 2D numpy array same size as the input image.

    TODO: Implement the convolve_image function here.
    '''
    H, W = image.shape
    Hk, Wk = filter_matrix.shape
    im_pad = np.pad(im, Hk//2, mode='constant')  # pad the border of the original image
    out = np.zeros_like(image)
    filter_matrix = np.flip(filter_matrix)
    for i in range(H):
      for j in range(W):
        out[i,j] = np.sum(im_pad[i:i+Hk, j:j+Wk] * filter_matrix)
    return out

Problem 1c: Mean Filter with Convolution
----------------------------

Implement this same operation with a convolution instead. Fill in the mean filter matrix here, and visualize the convolution result.

In [ ]:
mean_filt = 1/121 * np.ones((11,11))
''' TODO: Create a mean filter matrix here. '''

Apply mean filter convolution using your `convolve_image` function and the `mean_filt` matrix.

In [ ]:
my_implement_img = convolve_image(im, mean_filt)
show_image(convolve_image(im, mean_filt))

Compare your convolution result with the `scipy.signal.convolve2d` function (they should look the same).

In [ ]:
builtin_implementation = convolve2d(im, mean_filt,mode='same')
show_image(convolve2d(im, mean_filt))

Note: In the sections below, we will use the `scipy.signal.convolve2d` function for grading. But fill free to test your `convolve_image` function on other filters as well.

In [ ]:
np.allclose(my_implement_img, builtin_implementation)

Problem 1d: Gaussian Filter
---------------

Instead of using a mean filter, let's use a Gaussian filter. Create a 2D Gaussian filter, and plot the result of the convolution.

Hint: You can first construct a one dimensional Gaussian, then use it to create a 2D dimensional Gaussian.

In [ ]:
def gaussian_filter(sigma, k=20):
    '''
    Args:
        sigma: the standard deviation of Gaussian kernel.
        k: controls size of the filter matrix.
    Returns:
        a 2D Gaussian filter matrix of the size (2k+1, 2k+1).

    TODO: Implement a Gaussian filter here.
    '''
    size = 2*k+1
    out = np.zeros((size,size))
    x, y = np.meshgrid(np.arange(size), np.arange(size))  # 创建坐标网格
    out = np.exp(-((x-k)**2 + (y-k)**2)/(2*sigma**2))
    out /= np.sum(out)
    return out



show_image(convolve2d(im, gaussian_filter(2)))

The amount the image is blurred changes depending on the sigma parameter. Change the sigma parameter to see what happens. Try a few different values.

In [ ]:
show_image(convolve2d(im, gaussian_filter(5)))

Problem 1e: Visualizing Gaussian Filter
---------------------------
Try changing the sigma parameter below to visualize the Gaussian filter directly. This gives you an idea of how different sigma values create different convolved images.

In [ ]:
plt.imshow(gaussian_filter(sigma=5))

Problem 2: Edge Detection
=========================

There are a variety of filters that we can use for different tasks. One such task is edge detection, which is useful for finding the boundaries regions in an image. In this part, your task is to use convolutions to find edges in images. Let's first load up an edgy image.

In [ ]:
im = load_image('edge_detection_image.jpg')
im = im.mean(axis=2) # convert to grayscale
show_image(im)

Problem 2a: Image Gradient Filters
----------------------------------

Implement a filter to detect gradients, and convolve it with the image.  Show the result.

In [ ]:
# 什么filter可以detect gradient？
# 对一个image 求x,y 的gradient相当于和某个filter做卷积
filt = np.array([
    [-1,0,1],
    [-2,0,2],
    [-1,0,1]
])
''' TODO: Implement filter here. '''

plt.imshow(convolve2d(im, filt), cmap='gray')

Noise
-----

The issue with the basic gradient filters is that it is sensitive to noise in the image. Let's add some Gaussian noise to the image below, and visualize what happens. The edges should be hard to see.

In [ ]:
im = load_image('edge_detection_image.jpg')
im = im.mean(axis=2)
im = im + 0.2*np.random.randn(*im.shape)

f, axarr = plt.subplots(1,2)
axarr[0].imshow(im, cmap='gray')
axarr[1].imshow(convolve2d(im, filt), cmap='gray')

Problem 2b: Laplacian Filters
-----------------

Laplacian filters are edge detectors that are robust to noise (Why is this? Think about how the filter is constructed.). Implement a Laplacian filter below for both horizontal and vertical edges.

In [ ]:
# second derivative

lap_x_filt = np.array([[ 1,  0, -1],
                                  [ 2, -4,  2],
                                  [ 1,  0, -1]])
''' TODO: Implement a Laplacian filter for horizontal edges. '''
lap_y_filt = np.array([[ 1,  2,  1],
                                [ 0, -4,  0],
                                [-1, -2, -1]])
''' TODO: Implement a Laplacian filter for vertical edges. '''

f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(convolve2d(im, lap_y_filt), cmap='gray')
axarr[0,1].imshow(convolve2d(im, lap_x_filt), cmap='gray')
axarr[1,0].imshow(lap_y_filt, cmap='gray')
axarr[1,1].imshow(lap_x_filt, cmap='gray')

Problem 3: Hybrid Images
========================

Hybrid images is a technique to combine two images in one. Depending on the distance you view the image, you will see a different image. This is done by merging the high-frequency components of one image with the low-frequency components of a second image. In this problem, you are going to use the Fourier transform to make these images. But first, let's visualize the two images we will merge together.

In [ ]:
from numpy.fft import fft2, fftshift, ifftshift, ifft2

dog = load_image('dog.jpg').mean(axis=-1)[:, 25:-24]
cat = load_image('cat.jpg').mean(axis=-1)[:, 25:-24]

f, axarr = plt.subplots(1,2)
axarr[0].imshow(dog, cmap='gray')
axarr[1].imshow(cat, cmap='gray')

Problem 3a: Fourier Transform
-----------------

In the code box below, compute the Fourier transform of the two images. You can use the fft2 function. You can also use the fftshift function, which may help in the next section.

In [ ]:
from scipy.fft import fft2, fftshift

cat_fft = fftshift(fft2(cat))
''' TODO: compute the Fourier transform of the cat. '''
dog_fft = fftshift(fft2(dog))
''' TODO: compute the Fourier transform of the dog. '''

# Visualize the magnitude and phase of cat_fft. This is a complex number, so we visualize
# the magnitude and angle of the complex number.
# Curious fact: most of the information for natural images is stored in the phase (angle).
f, axarr = plt.subplots(1,2)
# 计算傅立叶变换的幅值（并取对数增强对比度）
axarr[0].imshow(np.log(np.abs(cat_fft)), cmap='gray')
# 计算傅立叶变换的相位信息
axarr[1].imshow(np.angle(cat_fft), cmap='gray')

Problem 3b: Low and High Pass Filters
-------------------------

By masking the Fourier transform, you can compute both low and high pass of the images. In Fourier space, write code below to create the mask for a high pass filter of the cat, and the mask for a low pass filter of the dog. Then, convert back to image space and visualize these images.

You may need to use the functions ifft2 and ifftshift.

In [ ]:
# high_mask = None   ''' TODO: Create the mask for a high pass filter of the cat. '''
# low_mask = None   ''' TODO: Create the mask for a low pass filter of the dog. '''

# cat_filtered = None  ''' TODO: Apply the high pass filter on the cat and convert back to image space. '''
# dog_filtered = None  ''' TODO: Apply the low pass filter on the dog and convert back to image space. '''

# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(dog_filtered, cmap='gray')
# axarr[1].imshow(cat_filtered, cmap='gray')


from scipy.fft import fft2, ifft2, fftshift, ifftshift

def create_circular_mask(shape, radius):
    """
    生成一个圆形掩码
    :param shape: (H, W) 形状
    :param radius: 圆的半径
    :return: 二值掩码，圆内为1，圆外为0
    """
    H, W = shape
    Y, X = np.ogrid[:H, :W]
    center = (H // 2, W // 2)
    mask = ((X - center[1])**2 + (Y - center[0])**2) <= radius**2
    return mask

# 生成高通和低通滤波掩码
shape = cat.shape
radius = 30  # 调整半径以改变滤波强度
low_pass_mask = create_circular_mask(shape, radius)
high_pass_mask = ~low_pass_mask

# 应用掩码
cat_high_pass = cat_fft * high_pass_mask
dog_low_pass = dog_fft * low_pass_mask

# 进行 ifftshift 再转换回时域
cat_filtered = np.abs(ifft2(ifftshift(cat_high_pass)))
dog_filtered = np.abs(ifft2(ifftshift(dog_low_pass)))

# 显示结果
fig, axarr = plt.subplots(2, 2, figsize=(12, 8))

# 原始图像
axarr[0, 0].imshow(cat, cmap='gray')
axarr[0, 0].set_title("Original Cat")
axarr[0, 0].axis("off")

axarr[1, 0].imshow(dog, cmap='gray')
axarr[1, 0].set_title("Original Dog")
axarr[1, 0].axis("off")

# 低通滤波后的狗
axarr[1, 1].imshow(dog_filtered, cmap='gray')
axarr[1, 1].set_title("Low-pass Filtered Dog")
axarr[1, 1].axis("off")

# 高通滤波后的猫
axarr[0, 1].imshow(cat_filtered, cmap='gray')
axarr[0, 1].set_title("High-pass Filtered Cat")
axarr[0, 1].axis("off")

plt.show()


Problem 3c: Hybrid Image Results
--------------------

Now that we have the high pass and low pass fitlered images, we can create a hybrid image by adding them. Write the code to combine the images below, and visualize the hybrd image.

Depending on whether you are close or far away from your monitor, you should see either a cat or a dog.  Try creating a few different hybrid images from your own photos or photos you found. Submit them, and we will show the coolest ones in class.

In [ ]:
hybrid = cat_filtered + dog_filtered

''' TODO: Compute the hybrid image here. '''

plt.imshow(hybrid, cmap='gray')

Acknowledgements
----------------

This homework is based on assignments from Aude Oliva at MIT, and James Hays at Georgia Tech.